In [1]:
import os
import numpy as np
import h5py
import glob
import argparse
import sys

# ROOT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(ROOT_DIR)

from contact_graspnet.data import PointCloudReader
from contact_graspnet.mesh_utils import in_collision_with_gripper, grasp_contact_location

import pprint as pp 

parser = argparse.ArgumentParser(description="Grasp data reader")
parser.add_argument('root_folder', help='Root dir with grasps, meshes, mesh_contacts and splits', type=str)
parser.add_argument('--num_grasp_scenes', type=int, default=10000)
parser.add_argument('--splits','--list', nargs='+')
parser.add_argument('--max_iterations', type=int, default=100)
parser.add_argument('--gripper_path', type=str, default='gripper_models/panda_gripper/panda_gripper.obj')
parser.add_argument('--min_num_objects', type=int, default=8)
parser.add_argument('--max_num_objects', type=int, default=12)
parser.add_argument('--start_index', type=int, default=0)
parser.add_argument('--load_existing', type=str, default=None)
parser.add_argument('--output_dir', type=str, default='scene_contacts')
parser.add_argument('-vis', action='store_true', default=False)
args = parser.parse_args(['/home/tele_mani_u20/code_proj/cgn/June_4/contact_graspnet/acronym'])

root_folder = args.root_folder
splits = args.splits if args.splits else ['train']
max_iterations = args.max_iterations
gripper_path = args.gripper_path
number_of_scenes = args.num_grasp_scenes
min_num_objects = args.min_num_objects
max_num_objects = args.max_num_objects
start_index = args.start_index
load_existing = args.load_existing
output_dir = args.output_dir
visualize = args.vis


In [2]:
from tools.create_table_top_scenes import TableScene

table_scene = TableScene(root_folder, gripper_path, splits=splits)
table_scene._scene_count = start_index

table_scene.reset()

In [9]:
import random
from tools.create_table_top_scenes import load_mesh

max_iter = 100 

num_objects = np.random.randint(min_num_objects,max_num_objects+1)
# Begin table_scene.arrange
# scene_grasps, scene_contacts, obj_paths, obj_transforms, obj_scales, obj_grasp_idcs = \
#     table_scene.arrange(num_objects, max_iterations)

num_obj = num_objects

self = table_scene

self._table_pose[2,3] -= self._lower_table
self.add_object('table', self.table_mesh, self._table_pose)       

self._support_objects.append(self.table_support)    

obj_paths = []
obj_transforms = []
obj_scales = []
grasp_paths = []

for i in range(num_obj):
    # obj_mesh, random_grasp_path = self.get_random_object()

    while True:
        random_category = random.choice(self.category_list)
        cat_obj_paths = [obj_p for split in self.splits for obj_p in self.data_splits[random_category][split]]
        if cat_obj_paths:
            random_grasp_path = random.choice(cat_obj_paths)
            if random_grasp_path in self.contact_infos:
                break

    mesh_root_dir = self.root_folder
    filename = os.path.join(self.root_folder, 'grasps', random_grasp_path)
    scale = None 

    data = h5py.File(filename, "r")
    mesh_fname = data["object/file"][()]
    mesh_scale = data["object/scale"][()] if scale is None else scale

    try:
        obj_mesh = \
            load_mesh(
                os.path.join(self.root_folder, 'grasps', random_grasp_path), 
                self.root_folder
            )
    
    # For mesh that does not exist
    except ValueError:
        continue
    
    mesh_mean =  np.mean(obj_mesh.vertices, 0, keepdims=True)
    obj_mesh.vertices -= mesh_mean
    
    try:
        success, placement_T = self.find_object_placement(obj_mesh, max_iter)
    except Exception as exc: 
        print(exc, random_grasp_path, " after {} seconds!".format(time_out))
        continue

    if success:
        self.add_object(random_grasp_path, obj_mesh, placement_T)

        obj_scales.append(float(random_grasp_path.split('_')[-1].split('.h5')[0]))
        obj_paths.append(os.path.join('meshes', '/'.join(random_grasp_path.split('_')[:2]) + '.obj'))
        obj_transforms.append(placement_T)

        grasp_paths.append(random_grasp_path)

    else:
        print("Couldn't place object", random_grasp_path, " after {} iterations!".format(max_iter))

print('Placed {} objects'.format(len(obj_paths)))

Couldn't place object WallClock_e67f610cafeb2c0dbf93f50e43a43f7e_0.013414893327299164.h5  after 100 iterations!
Placed 9 objects
[[-0.09333803  0.15775127  0.98305775 -0.08299633]
 [-0.01491089 -0.98747888  0.15704499  0.1159742 ]
 [ 0.99552282  0.          0.09452155  0.35222337]
 [ 0.          0.          0.          1.        ]] Sideboard_1589ca6e3cdc1d4abd7834b955162bf5_0.0020315717019923326.h5


In [11]:
scene_filtered_grasps = []
scene_filtered_contacts = []
obj_grasp_idcs = []
grasp_count = 0

for obj_transform, grasp_path in zip(obj_transforms, grasp_paths):

    grasps, contacts = self.load_suc_obj_contact_grasps(grasp_path)

    # Transform grasps and contact points with the obj_transform.
    # The obj_transform is the transformation to place the object
    # on the table.
    transformed_grasps, transformed_contacts = self._transform_grasps(
        grasps, contacts, obj_transform)

    # Filter out grasps that are in collision with the scene
    filtered_grasps, filtered_contacts = \
        self._filter_colliding_grasps(transformed_grasps, transformed_contacts)
    
    scene_filtered_grasps.append(filtered_grasps)
    scene_filtered_contacts.append(filtered_contacts)

    grasp_count += len(filtered_contacts)
    obj_grasp_idcs.append(grasp_count)

scene_filtered_grasps = np.concatenate(scene_filtered_grasps,0)
scene_filtered_contacts = np.concatenate(scene_filtered_contacts,0)

self._table_pose[2,3] += self._lower_table
self.set_mesh_transform('table', self._table_pose)

In [12]:
scene_grasps, scene_contacts, obj_paths, obj_transforms, obj_scales, obj_grasp_idcs = \
    scene_filtered_grasps, scene_filtered_contacts, obj_paths, obj_transforms, obj_scales, obj_grasp_idcs

In [ ]:
table_scene.save_scene_grasps(output_dir, scene_grasps, scene_contacts, obj_paths, obj_transforms, obj_scales, obj_grasp_idcs)